In [48]:
!pip install deeppavlov

In [0]:
import deeppavlov

# Hybrid goal-oriented bot

## General architecture

<img src="https://github.com/deepmipt/dp_tutorials/blob/master/img/bot_architecture.png?raw=1" width="100%" />

Typical goal-oriented dialog system consists of three modules:


### 1. Natural Language Understanding

is responsible 
* for detecting the whole dialogue **domain**
  * _is it the restaurant reservation?_
  * _is it the weather forecasting?_,
* for detecting current user **intention**
  * _does user say hello?_
  * _is he/she asking the telephone number?_ and
* for extracting **named entities** in the last user utterane 
  * _did user specify type of food?_
  * _what date did he/she mention?_
    
Therefore, NLU may consist of three separate models: domain classifier, intent classifier and named entity recogniser (NER).
  
The NER model is also called a slot filler model. Because every output of NER can be converted to a semantic frame with slots. For example, suppose the output of the NER was the following:
  
  
      Is it windy     on weekends ?
     
      O  O  B-SUBTYPE O  B-DATE   O
    
then we can also convert it to a semantic frame:
 
     {
        "subtype": "windy",
        "date": "weekends"
      }


      
### 2. Dialogue Manager

is responsible
 * for deciding of the next **system action**.
   
It is often call a dialogue policy manager module. Suppose there are  four possible system actions:
  
  
    [welcome, request_place, inform_restaurant, bye]
    
Then on the current turn of the dialogue, the task of dialogue policy manager is to choose one of the four actions to do.

### 3. Natural Language Generation


is responsible for converting system action to a textual response to the user. The module can contain simple templates as well as generative models.

     welcome -> "Hello, how are you?"

## Our implementation

<img src="https://github.com/deepmipt/dp_tutorials/blob/master/img/bot_architecture2.png?raw=1" width="100%" />

The bot we'll use will consist of the following modules. 

**NLU** will contain
* a **NER model** (no domain or intent classifiers). The used NLU model is neural network based model introduced in the previous tutorial. 

The **Dialogue Manager** will consist of
* **a neural network** that takes input text representation and semantic frame and classifies over all possible system actions

The **NLG** module is
* **a template-based** module that maps actions to texts

## Dataset

**DatasetReader** is responsible for downloading dataset

In [50]:
from deeppavlov.dataset_readers.dstc2_reader import DSTC2DatasetReader

data = DSTC2DatasetReader().read('./dstc2')

2019-07-03 11:59:11.757 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from dstc2/dstc2-trn.jsonlist]
I0703 11:59:11.757528 140715737319296 dstc2_reader.py:112] [loading dialogs from dstc2/dstc2-trn.jsonlist]
2019-07-03 11:59:12.12 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from dstc2/dstc2-val.jsonlist]
I0703 11:59:12.012632 140715737319296 dstc2_reader.py:112] [loading dialogs from dstc2/dstc2-val.jsonlist]
2019-07-03 11:59:12.181 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from dstc2/dstc2-tst.jsonlist]
I0703 11:59:12.181758 140715737319296 dstc2_reader.py:112] [loading dialogs from dstc2/dstc2-tst.jsonlist]


DSTC2 (Dialogue State Tracking Challenge 2) data is now

* downloaded from web
* saved to ./dstc2

In [51]:
data['train'][1]

({'intents': [{'act': 'inform', 'slots': [['pricerange', 'cheap']]}],
  'text': 'cheap restaurant'},
 {'act': 'request_food', 'text': 'What kind of food would you like?'})

**DatasetIterator** is responsible for generating batches 



In [0]:
from overrides import overrides

from deeppavlov.core.common.registry import register
from deeppavlov.core.data.data_learning_iterator import DataLearningIterator


@register('my_dialog_iterator')
class MyDialogDatasetIterator(DataLearningIterator):

    @staticmethod
    def _dialogs(data):
        dialogs = []
        prev_resp_act = None
        for x, y in data:
            if x.get('episode_done'):
                del x['episode_done']
                prev_resp_act = None
                dialogs.append(([], []))
            x['prev_resp_act'] = prev_resp_act
            prev_resp_act = y['act']
            dialogs[-1][0].append(x)
            dialogs[-1][1].append(y)
        return dialogs

    @overrides
    def split(self, *args, **kwargs):
        self.train = self._dialogs(self.train)
        self.valid = self._dialogs(self.valid)
        self.test = self._dialogs(self.test)

In [53]:
iterator = MyDialogDatasetIterator(data)

x_dialog, y_dialog = iterator.train[0]
x_dialog

[{'intents': [], 'prev_resp_act': None, 'text': ''},
 {'intents': [{'act': 'inform', 'slots': [['pricerange', 'cheap']]}],
  'prev_resp_act': 'welcomemsg',
  'text': 'cheap restaurant'},
 {'intents': [{'act': 'inform', 'slots': [['this', 'dontcare']]}],
  'prev_resp_act': 'request_food',
  'text': 'any'},
 {'intents': [{'act': 'inform', 'slots': [['area', 'south']]}],
  'prev_resp_act': 'request_area',
  'text': 'south'},
 {'db_result': {'addr': 'cambridge leisure park clifton way cherry hinton',
   'area': 'south',
   'food': 'chinese',
   'name': 'the lucky star',
   'phone': '01223 244277',
   'postcode': 'c.b 1, 7 d.y',
   'pricerange': 'cheap'},
  'intents': [{'act': 'inform', 'slots': [['area', 'south']]}],
  'prev_resp_act': 'api_call',
  'text': 'south'},
 {'intents': [{'act': 'request', 'slots': [['slot', 'addr']]}],
  'prev_resp_act': 'inform_area+inform_food+offer_name',
  'text': 'address'},
 {'intents': [{'act': 'request', 'slots': [['slot', 'phone']]}],
  'prev_resp_act':

In [54]:
x_dialog[1]

{'intents': [{'act': 'inform', 'slots': [['pricerange', 'cheap']]}],
 'prev_resp_act': 'welcomemsg',
 'text': 'cheap restaurant'}

In [55]:
y_dialog[1]

{'act': 'request_food', 'text': 'What kind of food would you like?'}

## Using pretrained bot from deeppavlov

In [56]:
from deeppavlov import configs, build_model


bot = build_model(configs.go_bot.gobot_dstc2, download=True)

2019-07-03 11:59:14.173 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz download because of matching hashes
I0703 11:59:14.173046 140715737319296 download.py:116] Skipped http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz download because of matching hashes
2019-07-03 11:59:14.453 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/deeppavlov_data/gobot_dstc2_v7.tar.gz download because of matching hashes
I0703 11:59:14.453521 140715737319296 download.py:116] Skipped http://files.deeppavlov.ai/deeppavlov_data/gobot_dstc2_v7.tar.gz download because of matching hashes
2019-07-03 11:59:14.732 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/deeppavlov_data/slotfill_dstc2.tar.gz download because of matching hashes
I0703 11:59:14.732654 140715737319296 download.py:116] Skipped http://files.deeppavlov.ai/deeppavlov_data/slotfill_dstc2.tar.gz downlo

In [57]:
bot(['hi, i want some food'])

2019-07-03 12:00:00.25 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 472: Made api_call with {}, got 109 results.
I0703 12:00:00.025747 140715737319296 network.py:472] Made api_call with {}, got 109 results.


['The lucky star serves chinese food.']

In [58]:
bot(['I would like indian food instead'])

2019-07-03 12:00:00.108 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 472: Made api_call with {'food': 'indian'}, got 22 results.
I0703 12:00:00.108196 140715737319296 network.py:472] Made api_call with {'food': 'indian'}, got 22 results.


['Cocum serves indian food.']

In [0]:
# reset dialogue context
bot.reset()

## Training bot

To train a model in the deeppavlov library, you have to create it' **config file**.

Main section of the file are:

* dataset_reader
 * configuration of dataset reader component 
* data
 * download and saving to disk
* dataset_iterator
 * configuration of dataset iterator component generator of batches
* metadata
 * extra info 
 * urls for extra data download
 * telegram configuration
*train
 * training process configuration
 * size of batches
 * number of training epochs
*chainer
 * specifies data flow
 * which components are run and in what order


**chainer** consists of separate **Components**, which are independent classes that perform independent chunks of a model pipeline.

Our Components will be:
  
* a tokenizer that splits texts to separate words (or tokens)

In [0]:
tokenizer = {
    "class_name": "deeppavlov.models.go_bot.wrapper:DialogComponentWrapper",
    "component": { "class_name": "split_tokenizer" },
    "in": ["x"],
    "out": ["x_tokens"]
}

* a vocabulary of all words from the dataset (it's main goal is to convert tokens to indeces)

In [0]:
token_vocabulary = {
    "id": "token_vocab",
    "class_name": "simple_vocab",
    "fit_on": ["x_tokens"],
    "save_path": "{MODELS_PATH}/my_gobot/token.dict",
    "load_path": "{MODELS_PATH}/my_gobot/token.dict"
}

* a dialogue policy neural network with templates

In [67]:
network = {
    "in": ["x"],
    "in_y": ["y"],
    "out": ["y_predicted"],
    "main": True,
    "class_name": "go_bot",
    "load_path": "{MODELS_PATH}/my_gobot/model",
    "save_path": "{MODELS_PATH}/my_gobot/model",
    "debug": False,
    "word_vocab": "#token_vocab",
    "template_path": "{DOWNLOADS_PATH}/dstc2/dstc2-templates.txt",
    "template_type": "DualTemplate",
    "api_call_action": "api_call",
    "use_action_mask": False,
    "network_parameters": {
      "learning_rate": 0.005,
      "dropout_rate": 0.5,
      "l2_reg_coef": 7e-4,
      "hidden_size": 128,
      "dense_size": 160
    },
    "slot_filler": None,
    "intent_classifier": None,
    "embedder": None,
    "bow_embedder": {
      "class_name": "bow",
      "depth": "#token_vocab.__len__()",
      "with_counts": True
    },
    "slot_filler": {
      "config_path": "{DEEPPAVLOV_PATH}/configs/ner/slotfill_dstc2.json"
    }
    "tokenizer": {
      "class_name": "stream_spacy_tokenizer",
      "lowercase": False
    },
    "tracker": {
      "class_name": "featurized_tracker",
      "slot_names": ["pricerange", "this", "area", "food", "name"]
    }
}

SyntaxError: ignored

As you can see important parameters of a **Component** are:

* name
 * registered name of the component
 * *it is a link to python component implementation*
* save_path
 * path to save the component
 * *sometimes is optional, for example, for tokenizers*
* load_path
 * path to load the component 
 * *sometimes is optional, for example, for tokenizers*

#### NER model

We will also use a pretrained NER model:

In [68]:
ner_model = build_model(deeppavlov.configs.ner.slotfill_dstc2, download=True)

ner_model(['i want cheap food in chinese reastaurant in the south of town'])

2019-07-03 12:03:30.527 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/deeppavlov_data/slotfill_dstc2.tar.gz download because of matching hashes
I0703 12:03:30.527128 140715737319296 download.py:116] Skipped http://files.deeppavlov.ai/deeppavlov_data/slotfill_dstc2.tar.gz download because of matching hashes
2019-07-03 12:03:30.535 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from /root/.deeppavlov/models/slotfill_dstc2/word.dict]
I0703 12:03:30.535531 140715737319296 simple_vocab.py:103] [loading vocabulary from /root/.deeppavlov/models/slotfill_dstc2/word.dict]
2019-07-03 12:03:30.544 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from /root/.deeppavlov/models/slotfill_dstc2/tag.dict]
I0703 12:03:30.544561 140715737319296 simple_vocab.py:103] [loading vocabulary from /root/.deeppavlov/models/slotfill_dstc2/tag.dict]
2019-07-03 12:03:31.583 INFO in 'd

[{'area': 'south', 'food': 'chinese', 'pricerange': 'cheap'}]

In [0]:
# free memory
del ner_model

### Full pipeline config

In [0]:
basic_config = {
  "dataset_reader": {
    "class_name": "dstc2_reader",
    "data_path": "{DOWNLOADS_PATH}/dstc2"
  },
  "dataset_iterator": {
    "class_name": "my_dialog_iterator"
  },
  "chainer": {
    "in": ["x"],
    "in_y": ["y"],
    "out": ["y_predicted"],
    "pipe": [
        tokenizer,
        token_vocabulary,
        network
    ]
  },
  "train": {
    "epochs": 200,
    "batch_size": 4,

    "metrics": ["per_item_dialog_accuracy"],
    "validation_patience": 10,
    
    "val_every_n_batches": 15,
    "val_every_n_epochs": -1,

    "log_every_n_batches": 15,
    "log_every_n_epochs": -1,
    "show_examples": False,
    "validate_best": True,
    "test_best": True
  },
  "metadata": {
    "variables": {
      "ROOT_PATH": "~/.deeppavlov",
      "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
      "MODELS_PATH": "./models",
      "CONFIGS_PATH": "./configs"
    },
    "requirements": [
      "{DEEPPAVLOV_PATH}/requirements/tf.txt",
      "{DEEPPAVLOV_PATH}/requirements/fasttext.txt",
      "{DEEPPAVLOV_PATH}/requirements/spacy.txt",
      "{DEEPPAVLOV_PATH}/requirements/en_core_web_sm.txt"
    ],
    "labels": {
      "telegram_utils": "GoalOrientedBot",
      "server_utils": "GoalOrientedBot"
    },
    "download": [
      {
        "url": "http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz",
        "subdir": "{DOWNLOADS_PATH}/dstc2"
      }
    ]
  }
}

In [0]:
from deeppavlov import train_model, build_model

In [0]:
bot = train_model(basic_config, download=True)

2019-07-03 12:03:42.928 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz download because of matching hashes
I0703 12:03:42.928655 140715737319296 download.py:116] Skipped http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz download because of matching hashes
2019-07-03 12:03:42.932 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /root/.deeppavlov/downloads/dstc2/dstc2-trn.jsonlist]
I0703 12:03:42.932081 140715737319296 dstc2_reader.py:112] [loading dialogs from /root/.deeppavlov/downloads/dstc2/dstc2-trn.jsonlist]
2019-07-03 12:03:43.193 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /root/.deeppavlov/downloads/dstc2/dstc2-val.jsonlist]
I0703 12:03:43.193427 140715737319296 dstc2_reader.py:112] [loading dialogs from /root/.deeppavlov/downloads/dstc2/dstc2-val.jsonlist]
2019-07-03 12:03:43.375 INFO in 'deeppavlov.da

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0035}, "time_spent": "0:00:13", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 4, "metrics": {"per_item_dialog_accuracy": 0.1935}, "time_spent": "0:00:14", "epochs_done": 0, "batches_seen": 15, "train_examples_seen": 60, "learning_rate": 0.005, "momentum": null, "loss": 2.0850544850031536}}


2019-07-03 12:04:11.327 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.1643
I0703 12:04:11.327934 140715737319296 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.1643
2019-07-03 12:04:11.330 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0703 12:04:11.330149 140715737319296 nn_trainer.py:166] Saving model
2019-07-03 12:04:11.336 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /content/models/my_gobot/model]
I0703 12:04:11.336540 140715737319296 tf_model.py:76] [saving model to /content/models/my_gobot/model]
2019-07-03 12:04:11.496 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 701: [saving parameters to /content/models/my_gobot/model.json]
I0703 12:04:11.496610 140715737319296 network.py:701] [saving parameters to /content/models/my_gobot/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.1643}, "time_spent": "0:00:26", "epochs_done": 0, "batches_seen": 15, "train_examples_seen": 60, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 4, "metrics": {"per_item_dialog_accuracy": 0.4516}, "time_spent": "0:00:27", "epochs_done": 0, "batches_seen": 30, "train_examples_seen": 120, "learning_rate": 0.005, "momentum": null, "loss": 1.52786705493927}}


2019-07-03 12:04:24.191 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.3105
I0703 12:04:24.191653 140715737319296 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.3105
2019-07-03 12:04:24.193 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0703 12:04:24.193655 140715737319296 nn_trainer.py:166] Saving model
2019-07-03 12:04:24.196 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /content/models/my_gobot/model]
I0703 12:04:24.196473 140715737319296 tf_model.py:76] [saving model to /content/models/my_gobot/model]
2019-07-03 12:04:24.359 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 701: [saving parameters to /content/models/my_gobot/model.json]
I0703 12:04:24.359417 140715737319296 network.py:701] [saving parameters to /content/models/my_gobot/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.3105}, "time_spent": "0:00:39", "epochs_done": 0, "batches_seen": 30, "train_examples_seen": 120, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 4, "metrics": {"per_item_dialog_accuracy": 0.4412}, "time_spent": "0:00:40", "epochs_done": 0, "batches_seen": 45, "train_examples_seen": 180, "learning_rate": 0.005, "momentum": null, "loss": 1.138824737071991}}


2019-07-03 12:04:37.109 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.325
I0703 12:04:37.109192 140715737319296 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.325
2019-07-03 12:04:37.111 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0703 12:04:37.111557 140715737319296 nn_trainer.py:166] Saving model
2019-07-03 12:04:37.115 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /content/models/my_gobot/model]
I0703 12:04:37.115328 140715737319296 tf_model.py:76] [saving model to /content/models/my_gobot/model]
2019-07-03 12:04:37.285 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 701: [saving parameters to /content/models/my_gobot/model.json]
I0703 12:04:37.285230 140715737319296 network.py:701] [saving parameters to /content/models/my_gobot/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.325}, "time_spent": "0:00:52", "epochs_done": 0, "batches_seen": 45, "train_examples_seen": 180, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 4, "metrics": {"per_item_dialog_accuracy": 0.4048}, "time_spent": "0:00:53", "epochs_done": 0, "batches_seen": 60, "train_examples_seen": 240, "learning_rate": 0.005, "momentum": null, "loss": 0.986410895983378}}


In [0]:
# or load trained bot from disk
bot = build_model(basic_config)

In [0]:
bot(['hi, i want some cheap food'])

In [0]:
bot(['bye'])

In [0]:
# free memory
del bot